In [ ]:
# The purpose of this code is to develop a GIF with a map of Minnesota (MN) successively adding MN state highways-just for fun

In [142]:
pip install imageio

     |████████████████████████████████| 3.3 MB 2.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [143]:
import pandas as pd
import geopandas as gpd
import descartes
import matplotlib.pyplot as plt
import numpy as np
import shapely.geometry
import imageio # to make the GIF file

In [5]:
roads = 'STREETS_LOAD.shp'

In [80]:
states = '/Users/matthewwilliamhaas/Documents/shapefile_data_sources/cb_2018_us_county_20m/cb_2018_us_county_20m.shp'
map_states = gpd.read_file(states) # read in counties

In [83]:
MN = map_states[(map_states.STATEFP == '27')]
MN = MN.to_crs({'init' : 'epsg:26915'})

In [6]:
# read in data
road_data = gpd.read_file(roads)

In [183]:
# subset data so we can map each highway independently
MN61 = road_data.STREET_NAM == 'MNTH 61'
MN36 = road_data.STREET_NAM == 'MNTH 36'
MN120 = road_data.STREET_NAM == 'MNTH 120'
MN280 = road_data.STREET_NAM == 'MNTH 280'

In [184]:
# needed to plot positions of highways
MN61_data = road_data[(MN61)].geometry
MN36_data = road_data[(MN36)].geometry
MN120_data = road_data[(MN120)].geometry
MN280_data = road_data[(MN280)].geometry

In [164]:
road_data[(MN280)]

,STREET_NAM,STREET_PRE,BASE_NAME,STREET_TYP,STREET_SUF,E_911,ROUTE_SYST,ROUTE_NUMB,TIS_CODE,DIVIDED_ON,TRAFFIC_DI,SHAPE_Leng,geometry
6306,MNTH 280,None,None,None,None,D,03,280,0300000280,M,I,565.695996,"LINESTRING (483915.876 4981782.748, 483866.013..."
6403,MNTH 280,None,None,None,None,D,03,280,0300000280,M,D,764.654286,"LINESTRING (483750.340 4982975.841, 483746.245..."
37938,MNTH 280,None,None,None,None,D,03,280,0300000280,M,D,189.294034,"LINESTRING (484100.094 4984125.000, 484043.563..."
40741,MNTH 280,None,None,None,None,D,03,280,0300000280,M,D,1026.468621,"LINESTRING (483940.625 4984029.888, 483909.872..."
40857,MNTH 280,None,None,None,None,D,03,280,0300000280,M,I,440.110534,"LINESTRING (483898.473 4978496.745, 483945.937..."
41691,MNTH 280,None,None,None,None,D,03,280,0300000280,M,I,193.678763,"LINESTRING (483954.751 4981171.896, 483955.062..."
104919,MNTH 280,None,None,None,None,D,03,280,0300000280,M,D,373.467493,"LINESTRING (483774.303 4982213.099, 483784.394..."
107875,MNTH 280,None,None,None,None,D,03,280,0300000280,M,D,150.525192,"LINESTRING (484003.531 4980385.999, 484005.281..."
115869,MNTH 280,None,None,None,None,D,03,280,0300000280,M,D,816.532840,"LINESTRING (483930.719 4981197.500, 483930.844..."
166658,MNTH 280,None,None,None,None,D,03,280,0300000280,M,D,67.053405,"LINESTRING (483760.190 4983042.158, 483750.678..."


In [69]:
# used this code for the map of MN and WI--not sure if it will be useful here or not
#TH61 = TH61.to_crs({"init" : "epsg:26915"})

In [182]:
# do the plotting
im = plt.imread('/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-36.png')
fig, ax = plt.subplots(1, figsize = (20,15))
MN.plot(color = 'white', linewidth = 1.0, ax = ax, edgecolor = 'black')
MN36_data.plot(color ='red', linewidth = 3.0, ax = ax, edgecolor = 'black')
#ax.plot(range(10))
newax = fig.add_axes([0.8, 0.5, 0.2, 0.3], anchor = 'NE', zorder = 1)
newax.imshow(im)
ax.axis('off') # turn off the axis
newax.axis('off')
plt.savefig('MN36.png')

In [180]:
# prevent plot from showing up below after execution of this code
%matplotlib agg
# load image to add to figure
im = plt.imread('/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-61.png')
# do the plotting
fig, ax = plt.subplots(1, figsize = (20,15))
MN.plot(color = 'white', linewidth = 1.0, ax = ax, edgecolor = 'black')
MN36_data.plot(color = 'grey', linewidth = 3.0, ax = ax, edgecolor = 'black')
MN61_data.plot(color = 'red', linewidth = 3.0, ax = ax, edgecolor = 'black')
#ax.plot(range(10))
newax = fig.add_axes([0.8, 0.5, 0.2, 0.3], anchor = 'NE', zorder = 1)
newax.imshow(im)
ax.axis('off') # turn off the axis
newax.axis('off')
plt.savefig('MN61.png')

In [190]:
# prevent plot from showing up below after execution of this code
%matplotlib agg
# do the plotting
im = plt.imread('/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-280.png')
fig, ax = plt.subplots(1, figsize = (20,15))
MN.plot(color = 'white', linewidth = 1.0, ax = ax, edgecolor = 'black')
MN36_data.plot(color ='grey', linewidth = 3.0, ax = ax, edgecolor = 'black')
MN61_data.plot(color = 'grey', linewidth = 3.0, ax = ax, edgecolor = 'black')
MN280_data.plot(color = 'red', linewidth = 3.0, ax = ax, edgecolor = 'black') # not showing up properly...
#ax.plot(range(10))
newax = fig.add_axes([0.8, 0.5, 0.2, 0.3], anchor = 'NE', zorder = 1)
newax.imshow(im)
ax.axis('off') # turn off the axis
newax.axis('off')
plt.savefig('MN280.png')

In [186]:
filenames = ['MN36.png', 'MN61.png', 'MN280.png']

In [191]:
# make the GIF file
with imageio.get_writer('movie.gif', mode = 'I', duration = 1) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)